In [1]:
# import dependencies
import numpy as np
import pandas as pd
import os

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.orm import Session

In [2]:
# define paths of clean files
new_hawaii_measurements_csv = os.path.join('Resources', 'clean_hawaii_measurements.csv')
new_hawaii_stations_csv = os.path.join('Resources', 'clean_hawaii_stations.csv') 

In [3]:
# create dataframe for stations
hi_stations = pd.read_csv(new_hawaii_stations_csv, index_col=0)
hi_stations.head(10)

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [4]:
# create dataframe for measurements
hi_measurements = pd.read_csv(new_hawaii_measurements_csv, index_col=0)

# convert date series to datetime data type
hi_measurements['date'] = pd.to_datetime(hi_measurements['date'], format="%Y/%m/%d")

hi_measurements.head(10)

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
5,USC00519397,2010-01-07,0.06,70
6,USC00519397,2010-01-08,0.00,64
7,USC00519397,2010-01-09,0.00,68
8,USC00519397,2010-01-10,0.00,73
9,USC00519397,2010-01-11,0.01,64
10,USC00519397,2010-01-12,0.00,61


In [5]:
# create stations class
class Stations(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)    

class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(DateTime)
    prcp = Column(Float)
    tobs = Column(Float)

In [6]:
# Establish Connection to database
engine = create_engine(f'sqlite:///hawaii.sqlite')

# Create both the Stations and Measurements tables within the database
Base.metadata.create_all(engine)

# create session object
session = Session(bind=engine)

In [7]:
# create lists to store measurement and station instances
stations = []

# create instances of stations class and add to stations list
for index, row in hi_stations.iterrows():
    
    
    station = Stations(station=row['station'], name=row['name'], 
                       latitude=row['latitude'], longitude=row['longitude'],
                       elevation=row['elevation'])

    stations.append(station)

# add stations instances to hawaii.sqlite database
session.bulk_save_objects(stations)

# commit changes
session.commit()

In [8]:
# create lists to store measurement and station instances
measurements = []

# create instances of stations class and add to stations list
for index, row in hi_measurements.iterrows():
    
    
    measurement = Measurements(station=row['station'], date=row['date'], 
                               prcp=row['prcp'], tobs=row['tobs'])

    measurements.append(measurement)

# add stations instances to hawaii.sqlite database
session.bulk_save_objects(measurements)

# commit changes
session.commit()

In [9]:
# print 1st 5 instances of stations table
stations = (session
            .query(Stations)
            .limit(5))

for station in stations:
    
    print(f'{station.station}\n{station.name}\n')

USC00519397
WAIKIKI 717.2, HI US

USC00513117
KANEOHE 838.1, HI US

USC00514830
KUALOA RANCH HEADQUARTERS 886.9, HI US

USC00517948
PEARL CITY, HI US

USC00518838
UPPER WAHIAWA 874.3, HI US



In [10]:
# print 1st 5 instances of measurements table
measurements = (session
                .query(Measurements)
                .limit(5))

for measurement in measurements:
    
    print(f'{measurement.station}\n{measurement.date}\n')

USC00519397
2010-01-01 00:00:00

USC00519397
2010-01-02 00:00:00

USC00519397
2010-01-03 00:00:00

USC00519397
2010-01-04 00:00:00

USC00519397
2010-01-07 00:00:00

